In [25]:

import requests
from datetime import datetime, timedelta
import pandas as pd
# Configuración
BASE_URL = "http://localhost:8000"

def predict_api(ticker, start_date, end_date, batch=False):
    url = f"{BASE_URL}/predict"
    payload = {
        "tick": ticker,
        "fecha_inicio": start_date,
        "fecha_fin": end_date,
        "batch": batch
    }
    response = requests.post(url, json=payload)
    return pd.DataFrame(response.json()["predicciones"].values(), response.json()["predicciones"].keys(), columns=["prediccion"])

In [ ]:
import glob
import os
import pandas as pd

# Obtener todos los archivos parquet en el directorio

modelos_disponibles = glob.glob("../models/*.joblib")
tickers = [i[i.find("prophet_") + 8: i.find("joblib")-1] for i in modelos_disponibles]



In [51]:
preds = predict_api(tickers[0], "2025-01-01", "2025-12-31", batch=False)

tickers_disponibles =[]
for ticker in tickers:  
    try:
        preds = predict_api(ticker, "2025-01-01", "2025-12-31", batch=False)
        tickers_disponibles.append(ticker)
    except:
        pass



In [ ]:
numero_variables = len(tickers_disponibles)

In [ ]:
preds = predict_api(tickers_disponibles[0], "2025-01-01", "2025-12-31", batch=False)
valor_inicial = preds.head(1).values[0][0]
valor_final = preds.tail(1).values[0][0]
preds.tail(1).values[0][0] - preds.head(1).values[0][0]

100 * valor_final / valor_inicial

In [ ]:
tickers_disponibles

In [ ]:
import pulp


# Variables de decisión
monto_invertido = pulp.LpVariable.dicts("monto_invertido", tickers_disponibles,lowBound=0)
variable_binaria = pulp.LpVariable.dicts("variable_binaria", tickers_disponibles,cat="Binary")

# Función objetivo

prob = pulp.LpProblem("optimizacion_platita", pulp.LpMaximize)
rendimiento_total = 0
for ticker in tickers_disponibles:
    preds = predict_api(ticker, "2025-01-01", "2026-12-31", batch=False)
    rendimiento = monto_invertido[ticker] * preds.tail(1).values[0][0] / preds.head(1).values[0][0]
    rendimiento_total += rendimiento

# Función objetivo
prob += rendimiento_total

# Restricciones
# Presupuesto
prob += pulp.lpSum(monto_invertido) <= 10000

# Diversificación
prob += pulp.lpSum(variable_binaria) >= 15

# Límite máximo por acción (20% del presupuesto)
for ticker in tickers_disponibles:
    prob += monto_invertido[ticker] <= 2000 * variable_binaria[ticker]  # 20% de 10000


# Restricciones de no negatividad
for ticker in tickers_disponibles:
    prob += monto_invertido[ticker] >= 0

# Conexión con la variable binaria
M = 10e10
for ticker in tickers_disponibles:
    prob += monto_invertido[ticker] <= M * variable_binaria[ticker]
    prob += monto_invertido[ticker] >= 100 * variable_binaria[ticker]

# Resolver el problema
prob.solve()


In [ ]:
prob.objective.value()

In [ ]:
print("\nVariables de productos:")
for ticker in tickers_disponibles:
    if pulp.value(variable_binaria[ticker]) == 1:
        print(f"{ticker} = {pulp.value(monto_invertido[ticker])}")

In [ ]:
preds = predict_api("S&P500", "2025-01-01", "2025-12-31", batch=False)
10000 *preds.tail(1).values[0][0] / preds.head(1).values[0][0]


In [ ]:
10000*(1.07)